# 인스타그램 크롤링 - Json 추출 방식

## 인스타그램 로그인 후 session_id, user_id 얻기

In [1]:
##### 하던 중이던거
## 인스타그램 로그인 후 session_id, user_id 를 얻는 과정.

# Chrome driver manager사용

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import ssl
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import pickle
# from fake_useragent import UserAgent

# 아마 https 관련 기능, requests 요청할 때 https 사이트의 경우 필요하다고 함.
ssl._create_default_https_context = ssl._create_unverified_context


user_id = "a"
session_id = "b"


## 이후에 def, if __name__ == '__main__' 방식으로 업데이트 예정
def executor():
    chrome_options = webdriver.ChromeOptions()
    # UserAgent 부분
#     userAgent = UserAgent().random
    userAgent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    chrome_options.add_argument(f'user-agent={userAgent}')
    chrome_options.add_argument('--incognito')
    return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)

# def exec_chrom():
#     options = Options()
#     userAgent = UserAgent().random
#     options.add_argument(f'user-agent={userAgent}')
#     options.add_argument('--incognito')
#     driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
#     driver.maximize_window()
#     driver.get(url)
#     return driver

def login(driver):
    global user_id, session_id
    
    # 02. 로그인
    #경로 지정 #동작 제어
    
    login_button = driver.find_element(by=By.XPATH,value = '//*[@class="_ab21"]/div[2]/div[3]/button[1]')
    login_button.click()
    print("click1")
    
    id_inp = driver.find_element(by=By.XPATH,value = '//*[@id="loginForm"]/div[1]/div[3]/div/label')
    id_inp.click()
    id_inp.send_keys("byeongu_son")
    print("click2")
    

    pw_inp = driver.find_element(by=By.XPATH,value = '//*[@id="loginForm"]/div[1]/div[4]/div/label')
    pw_inp.click()
    pw_inp.send_keys("asdf7530**")
    print("click3")
    

    log_btn = driver.find_element(by=By.XPATH,value = '//*[@id="loginForm"]/div[1]/div[6]/button')
    log_btn.click()
    
    time.sleep(4)
    pickle.dump(driver.get_cookies(), open("instagram_cookies.pkl", "wb")) #최초 로그인 후 쿠키 저장시에만
    cookies = pickle.load(open("instagram_cookies.pkl", "rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
        print(cookie)
        if(cookie['name'] == 'ds_user_id'):
            user_id = cookie['value']
        if(cookie['name'] == 'sessionid'):
            session_id = cookie['value']
            
    return user_id,session_id

    
def main():
    driver = executor()
    driver.set_window_size(414, 800) 	#브라우저 크기 414*800으로 고정
    driver.get('https://www.instagram.com') #인스타그램 웹 켜기
    driver.implicitly_wait(8) 	# 2초 대기
    
    user_id, session_id = login(driver)
    print(user_id)
    print(session_id)

main()


# 
# act = ActionChains(dr)      #동작 명령어 지정
# act.send_keys_to_element(id_box, 'rooneei').send_keys_to_element(password_box, 'asdf7530*').click(login_button).perform()     #아이디 입력, 비밀 번호 입력, 로그인 버튼 클릭 수행
# time.sleep(2)



[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.30M/6.30M [00:00<00:00, 12.5MB/s]


click1
click2
click3
{'domain': '.instagram.com', 'httpOnly': True, 'name': 'rur', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '"NAO\\0549678042146\\0541721214883:01f776dd0b7e7523c5a28682d7da3567a91c92d2783cf2c12d4d4c9c33133b0ffb02c076"'}
{'domain': '.instagram.com', 'expiry': 1690283683, 'httpOnly': True, 'name': 'shbid', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '"2588\\0549678042146\\0541721214881:01f7bd37bbee3d8462573596ad66ffaf47eb65f276fb6d45731932f15e87e0df987923c8"'}
{'domain': '.instagram.com', 'expiry': 1690283683, 'httpOnly': True, 'name': 'shbts', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '"1689678881\\0549678042146\\0541721214881:01f73926d95f58795588c8bd994cad76261ffb3db584d400f83ca0f475a4703897330973"'}
{'domain': '.instagram.com', 'expiry': 1721214883, 'httpOnly': True, 'name': 'sessionid', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '9678042146%3ANqurWxgTcMl8Xr%3A16%3AAYde7S-oVEu4l87T63ZCAl_cVN5_QMMxSxhY71t75g'}
{'

## 태그 검색했을 때 목록 추출 - 사람, 단체 구분가능

In [89]:
# 태그 관련한 검색어를 입력했을 때 목록들 - json parser 

import json
import urllib.request as req
from urllib.parse import quote
import requests

any_query = "후원" # 원하는 검색어
query = quote(any_query) 
# url = "https://www.instagram.com/api/v1/web/search/topsearch/?context=blended&include_reel=true&query=%ED%9B%84%EC%9B%90&rank_token=0.9534645826654842&search_surface=web_top_search"
url = "https://www.instagram.com/api/v1/web/search/topsearch/?context=blended&include_reel=true&query="+query+"&rank_token=0.9534645826654842&search_surface=web_top_search"
print(url)

header = {
    'Authority': 'www.instagram.com',
    'Method': 'GET',
    'Scheme': 'https',
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'X-Ig-App-Id': '936619743392459',
    'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"'
}


respons = requests.get(url, headers = header)
res = respons.json()
i=1
for user in res['users']:
    print(f"{i} : {user['user']['full_name']}")
    i+= 1

# res = req.urlopen(url)
# json_obj = json.load(res)
# i= 1;

# for user in json_obj['users']:
#     print(f"{i} : {user['user']['full_name']}")
#     i+= 1

https://www.instagram.com/api/v1/web/search/topsearch/?context=blended&include_reel=true&query=%ED%9B%84%EC%9B%90&rank_token=0.9534645826654842&search_surface=web_top_search
1 : 유기견 후원 브랜드 :: 버치포레스트
2 : 소아암 백혈병 아동 후원 브랜드 ‘러브무이’
3 : 유기견 후원 브랜드 🐶
4 : 유기동물 후원 브랜드
5 : 유기동물 후원 | 노바이노바이
6 : 후원
7 : 𝓪𝓵𝓲𝓴𝓪/𝓪𝓵𝓲𝔂
8 : 유기동물 후원 브랜드 매흐시
9 : 울산유기견*묘💛 봉사 후원
10 : 숟가락 하나씩 얹은 사람들의 유기견 후원 마켓
11 : 세이브캣 묘생역전 후원마켓
12 : 🐍🤍후원🖤🐍
13 : 세이브캣 후원 마켓
14 : 🍀방치견들 사료,간식 후원 주소 DM으로 알려드릴께요😊💕
15 : 𝑯𝑼𝑾𝑶𝑵.𝑮
16 : 공감쉼터 입양 홍보. 후원계좌
17 : huwon_dnl
18 : 𝙋𝙝𝙖̣𝙢 𝙃𝙪̛𝙤̛𝙣𝙜
19 : Hương Thu Thị
20 : 미혼모 후원 모금 계정
21 : Huwon Jeong
22 : Thu hương
23 : 육아인플루언서 밈채랜드
24 : 졸업작품 <인간미술관> 후원 계정✨
25 : 군산꽃집 후원(後園)꽃집
26 : 
27 : 후원
28 : 희원
29 : 아티스트 박정우, 안윤호의 학대아동 후원 프로젝트
30 : Hu J. W.
31 : 후원
32 : 후원내역 확인
33 : Ly Ly
34 : 위이어(WEYEAR), 사랑하는 시간💕l 보호종료. 자립준비. 후원. 펀딩. 기부 브랜드
35 : 제이미 여사 후원: http://toss.me/mrsjamie
36 : 지언이
37 : 조아오 후원
38 : Lê Hương Anh
39 : 👉개팔이 앵벌이 조심! 현명한 후원!!
40 : Thu Hươngg
41 : 인큐텐이수정 01055561668 후원01209116


# 태그를 입력했을 때, 나오는 게시물들에 관한 json

In [18]:
# 태그를 입력했을 때, 나오는 게시물들에 관한 json
# header 에서 2개 : Cookie(user_id, Session_id) 와 X-Ig-App-Id' 는 필수로 들어가야 함.)


import requests
from pprint import pprint

def content(tag_name,user_id,session_id):
    
#     tag_name = "후원"

    base_url = f"https://www.instagram.com/api/v1/tags/web_info/?tag_name={tag_name}"
    # base_url = "https://api.instagram.com/v1/users/self/media/recent/?access_token=12345"
    # headers = {'user-agent':'Mozilla/5.0 (iPhone; CPU iPhone OS 12_3_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 105.0.0.11.118 (iPhone11,8; iOS 12_3_1; en_US; en-US; scale=2.00; 828x1792; 165586599)'}

    headers = {
    #     'Authority': 'www.instagram.com',
    #     'Method': 'GET',
    #     'Scheme': 'https',
    #     'Accept': '*/*',
    #     'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    #     'Cookie' : 'ds_user_id=9678042146; sessionid=9678042146%3ANmAn8GxsZLltpC%3A28%3AAYdzwWnw6XGUaKGWUaDFe8SAc8g2NTtfXQmfh9kgyg;"',
        'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
    #     'Referer' : 'https://www.instagram.com/explore/tags/%ED%9B%84%EC%9B%90/',
    #     'Sec-Ch-Prefers-Color-Scheme': 'light',
    #     'Sec-Ch-Ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    #     'Sec-Ch-Ua-Full-Version-List': '"Not.A/Brand";v="8.0.0.0", "Chromium";v="114.0.5735.134", "Google Chrome";v="114.0.5735.134"',
    #     'Sec-Ch-Ua-Mobile': '?0',
    #     'Sec-Ch-Ua-Platform': '"Windows"',
    #     'Sec-Ch-Ua-Platform-Version': '"10.0.0"',
    #     'Sec-Fetch-Dest': 'empty',
    #     'Sec-Fetch-Mode': 'cors',
    #     'Sec-Fetch-Site': 'same-origin',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    #     'Viewport-Width': '842',
    #     'X-Asbd-Id': '129477',
    #     'X-Csrftoken': 'RSDZNwFUpElkUo9F6MO2M5HLCLj4D0PX',
        'X-Ig-App-Id': '936619743392459',
    #     'X-Ig-Www-Claim': 'hmac.AR220AKok4v1vxARXqZ6BbKkYf9kgjej9zQCryIlTk8LXig4',
    #     'X-Requested-With': 'XMLHttpRequest'
    }



    response= requests.get(base_url,headers=headers)

    # 만약 로그인이 끊기거나(api 50회정도가 문제인지, 시간이 문제인지는 모르겠음), 위의 조건 충족시키지 않을 시
    # 에러 페이지인 html 을 반환하게 되므로, resposne.json() 이 되지 않으면 response.text 를 통해 html 을 반환하는지 살펴볼 것.
    # response.text

    res = response.json()


    # 브라우저에 보이는 9개는 따로 json 파일의 최상단 중, data의 ['top']부분에 저장되어 있음.
    # ['sections'] 부분에 각각 3개씩 한묶음으로 해서 3개의 열이 json 파일로 보관되어 있음.

    all = res['count']
    print(all)
    root = res['data']['top']['sections'][0] # section 다음에 리스트 0~2(브라우저에 보이는 3열)
    tree1 = root['layout_content']['medias'] # medias 다음에 리스트 0~2(1열, 2열, 3열)

    # tree1 : 0~2
    # 코드는 각각에 대한 url -> instagram.com/p/{코드}
    i=1
    tag_user = ""
    for tree in tree1:
        tag_code = tree['media']['code']
        tag_text = tree['media']['caption']['text']
        tag_user = tree['media']['caption']['user']['username']
        print(i)
        print(f"내용 : {tag_text}")
        print(f"url : https://www.instgram.com/p/{tag_code}")
        print()
        base_url2 = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'

        response2= requests.get(base_url2,headers=headers)

        smartstore = response2.json()['data']['user']['external_url']
        print(f"{smartstore}")
        i+=1
    
#     base_url2 = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'
#     print(base_url2)
#     response2= requests.get(base_url2,headers=headers)
#     print(response2)
#     smartstore = response2.json()['data']['user']['external_url']
#     print(smartstore)

    print("-------------------------------------------------------")

    # 브라우저에 보이지 않는 나머지는 json 파일 중 끝쪽의 ['recent'] 부분에 저장되어 있음.
    # ({전체 개수} - {top 9개}) 를 각각 3개씩 한묶음으로 리스트의 ['sections'] 부분에 저장되어 있음.
    # {전체 개수} 에 도달할 때까지 위의 방식으로 나열되어 있다가, 그 수가 넘어서부터는 순서상관없이 데이터가 들어가있음.
    # -> 개수에 유의

    #if(count > 9) , 왠만하면 9개 넘을테니깐

    limit = all-9
    count = 0
    other_root=res['data']['recent']['sections'][0] # 여기까지 리스트 ({전체-9개}를 3으로 나눈 값만큼만 유효한 값)
    other_tree1=other_root['layout_content']['medias'] # 여기까지 리스트 (3개)

    # other_tree1 : 0~2
    i = 1
    for tree in other_tree1:
        # limit 넘는 순간 break!!!
        tag_text = tree['media']['caption']['text']
        print(i)
        print(tag_text)
        print()
        i+=1

    # for user in json_obj['users']:
    #     print(f"{i} : {user['user']['full_name']}")
    #     i+= 1
    # except Exception as e:
    #         print("getting user failed, due to '{}'".format(str(e)))


content("후원브랜드",user_id,session_id)

1532
1
내용 : 🦋🦋🦋 
FFM13 Silver925 Necklace
.
FFM13의 수익금의 일부는 생명의숲에 기부됩니다🌳🌏
.
#FFM13#모든일은나자신의믿음으로부터 
#기부브랜드 #후원브랜드 
#기부목걸이 #기부반지 #환경단체후원 
#나비목걸이 #나비반지
url : https://www.instgram.com/p/CuSIPV0JlRV

https://instabio.cc/3113014svmsmC
2
내용 : 🍋르몽마켓 12월 후원 내역 및 명단🍋
-
한국 백혈병 어린이 재단에 36,000원 기부하였습니다.

후원에 도움을 주신 모든 분들께 감사 드립니다🧡

🌅모두 새해 복 많이 받으세요😇
url : https://www.instgram.com/p/Cn00_-mSaRT

https://m.smartstore.naver.com/de_lemonmarket/products/6720658283
3
내용 : 따순해 유기묘 후원팔찌 흰색냥 리뷰가 도착했다냥🐾

정성스러운 후기와 예쁜 착용샷까지 감동이다냥🥹❤️🫶🏻

#따순해 #유기묘 #후원팔찌 #기부팔찌 #후원브랜드 #여름팔찌 #팔찌추천 #집사 #집사소통 #집사팔찌
url : https://www.instgram.com/p/CuPHGSzr5pp

https://url.kr/lavcgn
-------------------------------------------------------
1
개별각인 커스텀 반지🤍

나만의 문구로 더욱 의미있는 커스텀 각인 반지🫶🏻
외부와 내부, 원하시는 위치에 각인 가능해요 :)

다온 액세서리는 각인과 마감 퀄리티를 중요시해
모두 국내에서 수작업으로 제작되며
특별 각인 기법으로 높은 각인 퀄리티를 제공해요☺️

비가 많이 오는데 모두 빗길 조심하세요🙏🏻

2
유기동물들에게 행복한 하루를 선물해주셔서 감사드립니다!
⠀
퍼피띵은 아이들의
행복을 위해 안락사 없는
보호소 '내 사랑 바둑이'에
지속적으로 후원을 하고있습니다🐾
⠀
♥총 누적 후원금 8,012,000원♥
⠀
퍼피띵 프로젝트로

# Test

In [160]:
global kk
kk = "a"
ll = "b"

def value():
    global kk, ll
    a = {'domain': '.instagram.com', 'expiry': 1719682216, 'httpOnly': True, 'name': 'sessionid', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '9678042146%3AmgzAK4FHLqUtT3%3A26%3AAYeTZiyfz3hLq5Rzrj2L4K3C1L2RcD7-CF9vHEha9w'}
    b = {'domain': '.instagram.com', 'expiry': 1695922217, 'httpOnly': False, 'name': 'ds_user_id', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': '9678042146'}
    if(a['name'] == 'sessionid'):
        kk = a['value']
    if(b['name'] == 'ds_user_id'):
        ll = b['value']
    return kk,ll

def main():
    kk, ll = value()

    
main()
print(kk)
print(ll)

ds_user_id=asdf; sessionid=df;


In [ ]:
print(user_id)
print(session_id)

## 브랜드, 물품 별 데이터 추출 Test

### 브랜드 - bname(브랜드명), bintro(브랜드 설명), bimg(브랜드 이미지), ++ bcategory(후원하는 곳),

### 물품 - pname(물품명), pcategory(의류, 악세사리 등), pprice(물품 가격), pratio(할인율), pdiscount(할인 가격), pdelivery_info(배송정보), purl(물품 url), pimg(이미지), 

In [3]:
# 태그를 입력했을 때, 나오는 게시물들에 관한 json
# header 에서 2개 : Cookie(user_id, Session_id) 와 X-Ig-App-Id' 는 필수로 들어가야 함.)


import requests
from pprint import pprint

url_list =[]

headers = {
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
    'X-Ig-App-Id': '936619743392459',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
}

def ready(tag_name,user_id,session_id):
    
#     tag_name = "후원"

    base_url = f"https://www.instagram.com/api/v1/tags/web_info/?tag_name={tag_name}"
    headers = {
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'Cookie' : f'ds_user_id={user_id}; sessionid={session_id};"',
        'X-Ig-App-Id': '936619743392459',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    
    response= requests.get(base_url,headers=headers)

    res = response.json()

    # 브라우저에 보이는 9개는 따로 json 파일의 최상단 중, data의 ['top']부분에 저장되어 있음.
    # ['sections'] 부분에 각각 3개씩 한묶음으로 해서 3개의 열이 json 파일로 보관되어 있음.

    all_count = res['count']
    root = res['data']['top']['sections']#[0],[1],[2] # section 다음에 리스트 0~2(브라우저에 보이는 3열)
    # tree1 = root['layout_content']['medias'] # medias 다음에 리스트 0~2(1열, 2열, 3열)
    
    # 브라우저에 보이지 않는 부분들
    invisible_root=res['data']['recent']['sections']# [0] # 여기까지 리스트 ({전체-9개}를 3으로 나눈 값만큼만 유효한 값)
    # other_tree1=other_root['layout_content']['medias'] # 여기까지 리스트 (3개)
    
    return res,root,invisible_root

def check_redirect_contains(url, target_string):
    response = requests.get(url, allow_redirects=False)
    redirected_url = response.headers.get('Location')
    if redirected_url and target_string in redirected_url:
        return True
    return False


def list(res,root,invisible_root):
    all_count = res['count']
    print(f"전체 게시물 : {all_count}")
    i=1
    for view_list in root:
        for tree in view_list['layout_content']['medias']:
            print (f"------------{i}번째 게시물--------------")
            i+=1
            tag_code = tree['media']['code']
            tag_user = tree['media']['caption']['user']['username']
            print(f"크롤링 게시물 url : https://www.instgram.com/p/{tag_code}")
            
            # 게시물 올린 사람의 인스타 계정으로 이동
            user_url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'
            print(f"게시자 url : {user_url}")
            try:
                user_response= requests.get(user_url,headers=headers)
            except:
                main()
                user_response= requests.get(user_url,headers=headers)
            
            # 그 인스타 계정의 다른 외부 주소가 Naver Smart Store 인 경우에만 접속.
            external_url = user_response.json()['data']['user']['external_url']
            target_string = "smartstore.naver.com"
            
            if external_url and ( ( target_string in external_url ) or ( check_redirect_contains(external_url,target_string) ) ):
                if not (external_url in url_list):
                    print(f"Smart Store url : {external_url}")
                    url_list.append(external_url)
            else:
                print("XXXXXXXXXXXXXXXXXXXX", end = "\n\n\n")
                
    print("-----------=====-보이지 않는 게시물------=====------------")
    
    limit = all_count-9
    custom_limit = 40
    break_check = False
    count = 0
    for invisible_list in invisible_root:
        if(break_check):
            break
        for tree in invisible_list['layout_content']['medias']: # 여기까지 리스트 (3개)
            if ((i > limit) or (i> custom_limit)):
                break_check = True
                break
            else:
                print (f"------------{i}번째 게시물--------------")
                i+=1
                tag_code = tree['media']['code']
                tag_user = tree['media']['caption']['user']['username']
                print(f"크롤링 게시물 url : https://www.instgram.com/p/{tag_code}")

                # 게시물 올린 사람의 인스타 계정으로 이동
                user_url = f'https://www.instagram.com/api/v1/users/web_profile_info/?username={tag_user}'
                print(f"게시자 url : {user_url}")
                try:
                    user_response= requests.get(user_url,headers=headers)
                except:
                    main()
                    user_response= requests.get(user_url,headers=headers)

                # 그 인스타 계정의 다른 외부 주소가 Naver Smart Store 인 경우에만 접속.
                external_url = user_response.json()['data']['user']['external_url']
                target_string = "smartstore.naver.com"

                if external_url and ( ( target_string in external_url ) or ( check_redirect_contains(external_url,target_string) ) ):
                    if not (external_url in url_list):
                        print(f"Smart Store url : {external_url}")
                        url_list.append(external_url)
                else:
                    print("XXXXXXXXXXXXXXXXXXXX", end = "\n\n\n")
            


In [4]:
response,root,invisible_root = ready("후원브랜드",user_id,session_id)

list(response,root,invisible_root)

전체 게시물 : 1532
------------1번째 게시물--------------
크롤링 게시물 url : https://www.instgram.com/p/Cuazki5BRhQ
게시자 url : https://www.instagram.com/api/v1/users/web_profile_info/?username=ddasunhae
XXXXXXXXXXXXXXXXXXXX


------------2번째 게시물--------------
크롤링 게시물 url : https://www.instgram.com/p/CuSIPV0JlRV
게시자 url : https://www.instagram.com/api/v1/users/web_profile_info/?username=ffm13_official
XXXXXXXXXXXXXXXXXXXX


------------3번째 게시물--------------
크롤링 게시물 url : https://www.instgram.com/p/CellcOLBohJ
게시자 url : https://www.instagram.com/api/v1/users/web_profile_info/?username=jejusarang1004
XXXXXXXXXXXXXXXXXXXX


------------4번째 게시물--------------
크롤링 게시물 url : https://www.instgram.com/p/CoWh3IRPO00
게시자 url : https://www.instagram.com/api/v1/users/web_profile_info/?username=geee.yi
XXXXXXXXXXXXXXXXXXXX


------------5번째 게시물--------------
크롤링 게시물 url : https://www.instgram.com/p/CuZ1eYQJS3D
게시자 url : https://www.instagram.com/api/v1/users/web_profile_info/?username=ffm13_official
XXXXXXXXXXXXXXXX

In [151]:
url_list

['http://branddaon.com/',
 'https://m.smartstore.naver.com/de_lemonmarket/products/6720658283',
 'https://smartstore.naver.com/puffy_thing/',
 'https://m.smartstore.naver.com/heelaser',
 'https://smartstore.naver.com/myodeng']

In [5]:
url_list

['https://smartstore.naver.com/puffy_thing/',
 'https://m.smartstore.naver.com/de_lemonmarket/products/6720658283',
 'http://branddaon.com/',
 'https://m.smartstore.naver.com/heelaser',
 'https://smartstore.naver.com/myodeng']

# Reference 참고

In [ ]:
#01. 웹 열기
dr = webdriver.Chrome("C:/windows/chromedriver.exe") #웹드라이버로 크롬 웹 켜기
dr.set_window_size(414, 800) 	#브라우저 크기 414*800으로 고정
dr.get('https://www.instagram.com/') #인스타그램 웹 켜기
time.sleep(2) 	#2초 대기

# 02. 로그인
#경로 지정
id_box = dr.find_element_by_css_selector("#loginForm > div > div:nth-child(1) > div > label > input")   #아이디 입력창
password_box = dr.find_element_by_css_selector("#loginForm > div > div:nth-child(2) > div > label > input")     #비밀번호 입력창
login_button = dr.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')      #로그인 버튼

#동작 제어
act = ActionChains(dr)      #동작 명령어 지정
act.send_keys_to_element(id_box, '아이디').send_keys_to_element(password_box, '비밀번호').click(login_button).perform()     #아이디 입력, 비밀 번호 입력, 로그인 버튼 클릭 수행
time.sleep(2)

#03. 프로필 들어가기
#프로필 사진 클릭하기
my_photo = dr.find_element_by_css_selector('#react-root > section > nav > div._8MQSO.Cx7Bp > div > div > div.ctQZg.KtFt3 > div > div:nth-child(6) > div.EforU > span > img')
my_photo.click()
time.sleep(2)

#프로필 버튼 클릭하기
profile = dr.find_element_by_css_selector('section > nav > div._8MQSO.Cx7Bp > div > div > div.ctQZg > div > div:nth-child(6) > div.poA5q > div.uo5MA._2ciX.tWgj8.XWrBI > div._01UL2 > a:nth-child(1) > div')
profile.click()
time.sleep(2)

In [ ]:
## 인스타그램 로그인 후 session_id, user_id 구하는 (구)버전!

# Chrome driver manager사용

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
import ssl
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains


# 아마 https 관련 기능, requests 요청할 때 https 사이트의 경우 필요하다고 함.
ssl._create_default_https_context = ssl._create_unverified_context


## 이후에 def, if __name__ == '__main__' 방식으로 업데이트 예정
def executor():

  chrome_options = webdriver.ChromeOptions()
  return webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)


driver = executor()
driver.set_window_size(414, 800) 	#브라우저 크기 414*800으로 고정
driver.get('https://www.instagram.com/') #인스타그램 웹 켜기
time.sleep(2) 	#2초 대기

# 02. 로그인
#경로 지정 #동작 제어
id_inp = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label')
id_inp.click()
id_inp.send_keys("byeongu_son")

pw_inp = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label')
pw_inp.click()
pw_inp.send_keys("asdf7530**")

log_btn = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button')
log_btn.click()


# 
# act = ActionChains(dr)      #동작 명령어 지정
# act.send_keys_to_element(id_box, 'rooneei').send_keys_to_element(password_box, 'asdf7530*').click(login_button).perform()     #아이디 입력, 비밀 번호 입력, 로그인 버튼 클릭 수행
# time.sleep(2)
